In [9]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
import mysql.connector

# inisialisasi array untuk menampilkan hasil
id_detail = []
keterangan_lowongan = []
deskripsi_lowongan = []
stam_detail = []

# konektor ke database
db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='UnNgGrape'
        )
cursor = db.cursor()

cursor.execute("SELECT id_karir, link_lowongan FROM karir")
raw_karir = cursor.fetchall()
for row in raw_karir:
    id_karir = row[0]
    link = row[1]

    r = Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
    response = urlopen(r).read()
    soup = BeautifulSoup(response, "lxml")

    raw_detail = soup.find_all("div","b-opportunity-info__main")
    for p in raw_detail:

        # menggunakan try except karena ada beberapa estimasi keterangan yang tidak terlampir
        try:
            
            keterangan = p.find("div", "b-opportunity-info__main__left" ).get_text(separator=". ")
        except:
            keterangan = "-"
        
        # menggunakan try except karena ada beberapa deskripsi yang NoneType
        try:
            raw_deskripsi = p.find("div", "html-content b-opportunity-show__box" ).get_text(separator=". ")
            nltk_tokens = nltk.sent_tokenize(raw_deskripsi)
            deskripsi = ""

            for x in nltk_tokens:
                if x == "Disclaimer:.":
                    break
                deskripsi = deskripsi + " " + x
        except:
            deskripsi = "Eror: deskripsi Nonetype"

        raw_stam = keterangan + " " + deskripsi
        # create stemmer
        # nltk
        ps = PorterStemmer()
        # sastrawi
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        # stemming process
        # nltk
        words = word_tokenize(raw_stam)
        nltk_stammer = ""
        for w in words:
            nltk_stammer = nltk_stammer + " " + ps.stem(w)
        # sastrawi
        output_stammer = stemmer.stem(nltk_stammer)

        # mengecek apakah data sudah adat atau belum
        cursor.execute("SELECT id_karir FROM detail_karir WHERE id_karir = '" + str(id_karir) + "'")
        data = cursor.fetchall()
        # if else untuk pengecekan input data
        if data:
            print("Data exists")
        else:
            cursor.execute(
                        "INSERT INTO detail_karir(id_karir, keterangan_lowongan, deskripsi_lowongan, stam_detail)"
                        "VALUES ('"+ str(id_karir) +"', '"+ keterangan.replace("'", "").replace('"', '') +"', '"+ deskripsi.replace("'", "").replace('"', '') +"', '"+ output_stammer +"')"
                        )
            
            db.commit()
            print("Data has been added")

        id_detail.append(id_karir)
        keterangan_lowongan.append(keterangan)
        deskripsi_lowongan.append(deskripsi)
        stam_detail.append(output_stammer)

detail_dict ={'id':id_detail, 'keterangan':keterangan_lowongan, 'deskripsi':deskripsi_lowongan, 'stam':stam_detail}
df = pd.DataFrame(detail_dict, columns = ['id', 'keterangan', 'deskripsi', 'stam'])

df.sort_values('id', ascending=True)

Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added
Data has been added


,id,keterangan,deskripsi,stam
0,257,"Fungsi Kerja. Teknik, Sipil, Teknik, Mesin. Je...",Persyaratan. Lulusan minimal S1 dari Jurusan ...,fungsi kerja teknik sipil teknik mesin jenjang...
1,258,Fungsi Kerja. Konsultan. Jenjang Karir. Staf S...,Persyaratan. 1. Pendidikan S1 Tehnik Geodesi ...,fungsi kerja konsultan jenjang karir staf seni...
2,259,Fungsi Kerja. Konsultan. Jenjang Karir. Staf S...,Persyaratan. 1. Pria/Wanita\r\n. . 2. Bersedi...,fungsi kerja konsultan jenjang karir staf seni...
3,260,Fungsi Kerja. Konsultan. Jenjang Karir. Staf S...,Persyaratan. 1. Pria/ Wanita\r\n. . 2. Berpen...,fungsi kerja konsultan jenjang karir staf seni...
4,261,Fungsi Kerja. Administrasi. Jenjang Karir. Pem...,"Persyaratan. Wanita, Usia Max 25th. Pendidika...",fungsi kerja administrasi jenjang karir mula s...
...,...,...,...,...
128,385,"Fungsi Kerja. Teknik, Komputer, Teknik, Umum, ...",Persyaratan. 1. Candidate must possess at lea...,fungsi kerja teknik komput teknik umum teknik ...
129,386,"Fungsi Kerja. Teknik, Jaringan, TI, Network En...",Persyaratan. Minimum 2 years experience as Ne...,fungsi kerja teknik jaring ti network engin je...
130,387,"Fungsi Kerja. Teknik, Jaringan. Jenjang Karir....",Persyaratan. Min. D-3 dengan pengalaman minim...,fungsi kerja teknik jaring jenjang karir staf ...
131,388,"Fungsi Kerja. TI, Administrator, TI, Programme...",Persyaratan. Usia maksimal 35 tahun\r\n. Pend...,fungsi kerja ti administr ti programm ti syste...
